### Assignment 1
### NAME: RUHANI FAIHEEM RAHMAN
### STUDENT ID: 801126769

### Problem 1(Language Model Creation)(80points).

In  this  exercise,  you  will  train  probabilistic  language  models  to  distinguish between words in different languages. Rather than looking up whole words in a dictionary,  you  will  build  models  of  character  sequences  so  you  can  make  a guess about the language of unseen words. You will need to use NLTK and the Universal Declaration of Human Rights corpus.We  will  compare  across  different  languages  from  the  Universal  Declaration  of Human Rights documents. 

Use the following code to load the corpus and create sets of four languages.

In [1]:
import nltk 
from nltk.corpus import udhr

In [2]:
english = udhr.raw('English-Latin1') 
french = udhr.raw('French_Francais-Latin1') 
italian = udhr.raw('Italian_Italiano-Latin1') 
spanish = udhr.raw('Spanish_Espanol-Latin1')

### Train and Test Dataset

In [3]:
english_train, english_dev = english[0:1000], english[1000:1100] 
french_train, french_dev = french[0:1000], french[1000:1100]
italian_train, italian_dev = italian[0:1000], italian[1000:1100] 
spanish_train, spanish_dev = spanish[0:1000], spanish[1000:1100]  

english_test = udhr.words('English-Latin1')[0:1000] 
french_test = udhr.words('French_Francais-Latin1')[0:1000]
italian_test = udhr.words('Italian_Italiano-Latin1')[0:1000] 
spanish_test = udhr.words('Spanish_Espanol-Latin1')[0:1000]

### Model

#### Test of get_model

In [5]:
from nltk import word_tokenize
"""
    This method takes input text, preprocess the text, tokenize it and returns the list of ngram.
    
    Output looks like this
    [('u', 'n', 'i'), ('n', 'i', 'v'), ('i', 'v', 'e'), ('v', 'e', 'r'), ('e', 'r', 's'), ('r', 's', 'a'), ('s', 'a', 'l'), ('a', 'l', None), ...]
    
"""
def get_chars(text,n):
    text = text.lower()
    chars = list()
    a = word_tokenize(text)
    for a_ in a:
        b = list(nltk.ngrams(a_,n,pad_left='True',pad_right='True'))
        for b_ in b:
            chars.append(b_)
    return chars

#### Test for get_chars()

In [6]:
get_chars(english_train,3)[:10]

[(None, None, 'u'),
 (None, 'u', 'n'),
 ('u', 'n', 'i'),
 ('n', 'i', 'v'),
 ('i', 'v', 'e'),
 ('v', 'e', 'r'),
 ('e', 'r', 's'),
 ('r', 's', 'a'),
 ('s', 'a', 'l'),
 ('a', 'l', None)]

In [7]:
"""
    This helping method returns the pair list of the ngrams to prepare the data to use ConditionalFreqDist()
    Fist pair, Second pair
"""
def get_pair_list(char_lists, n):
    if n==1:
        pair_lists = char_lists
    elif n==2:
        pair_lists = (((w0), w1) for w0, w1 in char_lists)
    elif n==3:
        pair_lists = (((w0, w1), w2) for w0, w1, w2 in char_lists)
#     elif n==4:
#         pair_lists = (((w0, w1, w2), w3) for w0, w1, w2, w3 in char_lists)
#     elif n==5:
#         pair_lists = (((w0, w1, w2, w3), w4) for w0, w1, w2, w3, w4 in char_lists)
    else:
        print('We are not there yet!!!')
#    print(pair_lists)
    return pair_lists

In [8]:
"""
    This method returns the Frequency Distribution of the ngram list
"""
def get_distribution(pair_lists, n):
    if n==1:
        dist_lists = nltk.FreqDist(pair_lists)
    else:
        dist_lists = nltk.ConditionalFreqDist(pair_lists)
    return dist_lists

In [9]:
"""
    This method calculates the probability using add one smoothing formula
    
    For unigram, it uses simple N_gram rule to calculate probability
    For bigram and trigram, this method uses add one smoothing
    
"""
def predict_probabilities(word, model,help_model, n):
    word_chars = get_chars(word.lower(),n)
    word_ngrams  = list()

    if n == 1:
        for a in word_chars:
            word_ngrams.append(a[0])
    else:
        word_ngrams = word_chars

    probabilities = 1
    v = len(word_ngrams)
    
    """
        This section implemtns the add one smoothing for the ngram.
        
        formula for probabilities = (1+count(current))/(total_word+count(previous))
    """
    v = len(help_model)
    t_len = 1000
    for i in range(len(word_ngrams)):        
        curr = word_ngrams[i]
        
        if n == 1:
            key = 'None'
        elif n == 2:
            key = (curr[0])
        elif n == 3:
            key = (curr[0], curr[1])
        try:
            denomenator = help_model[key]
        except:
            denomenator = 0
        try:
            count = model[curr]
        except:
            count = 0

        if n==1:
            prob = (1+count)/(t_len)
        else:
            prob = (1.0+count)/(v+denomenator)
        probabilities *= prob
    return probabilities

In [10]:
"""
    This method calculated the ngram model. Returns Two list of Frequency Distribution.
    First list contains the frequency distribution of n grams
    Second list contains the frequency distribution of (n-1) grams
    
    If you call with n=3,
        first list contains [('o', 'f', None): 14.0, ('f', None, None): 16.0, (None, 'h', 'u'): 7.0, (None, 'h', 'a'): 5.0, (None, 'h', 'i'): 1.0, ...]
        second list contains [ ('e', 'a'): 9, ('a', 'm'): 3, ('m', 'b'): 2, ('b', 'l'): 2, ('l', 'e'): 6, ('e', None): 31, (None, 'w'): 11,]
    
"""
def get_model(train_data, n):
    total_chars = 1#len(train_data)
    ### Get chars list
    eng_chars = get_chars(train_data,n)
    
    ### get conditional pair list of char
    eng_tri_gram_pairs = get_pair_list(eng_chars,n)
    
    ### frequency distribution list
    eng_tri_gram_distribution = get_distribution(eng_tri_gram_pairs, n)
    
    eng_tri_gram_model = dict()
    conditional_model = dict()

    ### calculate model frequency for the training set
    for key, val in eng_tri_gram_distribution.items():
        if n==1:
            eng_tri_gram_model[key[0]] = eng_tri_gram_distribution[key]/total_chars

        elif n==2:
            for values in val:
                eng_tri_gram_model[(key,values)] = eng_tri_gram_distribution[key][values]/total_chars
                try:
                    conditional_model[key] = conditional_model[key] + eng_tri_gram_distribution[key][values]
                except:
                    conditional_model[key] = eng_tri_gram_distribution[key][values]
                
        elif n==3:
            for values in val:
                eng_tri_gram_model[(key[0],key[1],values)] = eng_tri_gram_distribution[key][values]/total_chars
                try:
                    conditional_model[key] = conditional_model[key] + eng_tri_gram_distribution[key][values]
                except:
                    conditional_model[key] = eng_tri_gram_distribution[key][values]
#         elif n==4:
#             for values in val:
#                 eng_tri_gram_model[(key[0],key[1],key[2],values)] = eng_tri_gram_distribution[key][values]/total_chars
#         elif n==5:
#             for values in val:
#                 eng_tri_gram_model[(key[0],key[1],key[2],key[3],values)] = eng_tri_gram_distribution[key][values]/total_chars
#    print(eng_tri_gram_model)
    return eng_tri_gram_model, conditional_model


In [11]:
a, b = get_model(english_train,3)

In [12]:
"""
This method is the main method here. This method will take train data, test data and ngram as parameters.
@parameters: two training data, two test data, ngram
"""
def model(train1, train2, test1, test2, n, des1, des2):
    """
    get_model with provide the model from the training data set. It returns a dictionary

    Dictionary looks like this

    {('e', 'c') 5.0,('e', 'a') 9.0,('e', None) 31.0,('e', 'n') 12.0,('e', 'q') 1.0,('e', 'm') 2.0,('e', 'e') 6.0, ...}
    
    ngram is the key and prediction is the value.
    """
    """
        ### calculate model for both training data set
    """
    english_trigram_model, help_model_1 = get_model(train1,n)
    french_trigram_model, help_model_2 = get_model(train2,n)

    """
        ### @pred_results stores the value for prediction
    """
    pred_results = {'false-positive':0,'false-negetive':0,'true-positive':0,'true-negetive':0}
 
    for words in test1:
        """
            Calculate probailities for each model
        """
        eng_pred = predict_probabilities(words, english_trigram_model,help_model_1,n)
        french_pred = predict_probabilities(words, french_trigram_model,help_model_2,n)
    
        if (eng_pred >= french_pred):
            pred_results['true-positive'] += 1
        else:
            pred_results['false-positive'] += 1
    """
        ### Accuracy of the first model.
    """
    print('Accuracy of the {} model for {} grams: {} %'.format(des1, n, pred_results['true-positive']/(pred_results['true-positive']+pred_results['false-positive']) * 100.0))

#     pred_results = {'false-positive':0,'false-negetive':0,'true-positive':0,'true-negetive':0}

#     for words in test2:
#         eng_pred = predict_probabilities(words, english_trigram_model,help_model_1,n)
#         french_pred = predict_probabilities(words, french_trigram_model,help_model_2,n)
#         if (eng_pred > french_pred):
#             pred_results['false-positive'] += 1
#         else:
#             pred_results['true-positive'] += 1

#     """
#         ### Accuracy of the second model.
#     """
#     print('Accuracy of the {} model for {} grams: {} %'.format(des2, n, pred_results['true-positive']/(pred_results['true-positive']+pred_results['false-positive']) * 100.0))
    return 

### Test Results

#### English Test Data with English vs Franch Model

In [13]:
model(english_train, french_train, english_test, english_test, 1, 'English','French')
model(english_train, french_train, english_test, english_test, 2, 'English','French')
model(english_train, french_train, english_test, english_test, 3, 'English','French')

Accuracy of the English model for 1 grams: 78.4 %
Accuracy of the English model for 2 grams: 87.8 %
Accuracy of the English model for 3 grams: 87.8 %


#### French Test Data with English vs Franch Model

In [14]:
model(french_train, english_train, french_test, french_test, 1, 'French', 'English')
model(french_train, english_train, french_test, french_test, 2, 'French', 'English')
model(french_train, english_train, french_test, french_test, 3, 'French', 'English')

Accuracy of the French model for 1 grams: 76.1 %
Accuracy of the French model for 2 grams: 64.60000000000001 %
Accuracy of the French model for 3 grams: 69.0 %


### Output Analysis

We can see here Accuracy of unigram for testing in English Data gets us 78.4% accuracy.
for BiGram, the accuracy reduces 87.8 %
for TriGram, the accuracy is also 87.8 %

On the other hand, The accuracy for French test data on French model is 76.1 for unigram, 64.6% for bigram and 69 % for trigram. Here unigram performances is not good at all. 

So, in this model testing, we can conclude that English can be easily distingusable.

### Problem 2** (Language Model Comparison) (20 points).

Perform  the  same  experiment as  above for  Spanish  vs.  Italian.  Which  language pair is harder to distinguish?

#### Italian Test Data with Spanish vs Italian Model

In [15]:
model(italian_train, spanish_train, italian_test, italian_test, 1, 'Italian', 'Spanish')
model(italian_train, spanish_train, italian_test, italian_test, 2, 'Italian', 'Spanish')
model(italian_train, spanish_train, italian_test, italian_test, 3, 'Italian', 'Spanish')

Accuracy of the Italian model for 1 grams: 65.2 %
Accuracy of the Italian model for 2 grams: 79.9 %
Accuracy of the Italian model for 3 grams: 78.9 %


#### Spanish Test Data with Spanish vs Italian Model

In [16]:
model(spanish_train, italian_train, spanish_test, spanish_test, 1, 'Spanish', 'Italian')
model(spanish_train, italian_train, spanish_test, spanish_test, 2, 'Spanish', 'Italian')
model(spanish_train, italian_train, spanish_test, spanish_test, 3, 'Spanish', 'Italian')

Accuracy of the Spanish model for 1 grams: 65.9 %
Accuracy of the Spanish model for 2 grams: 61.7 %
Accuracy of the Spanish model for 3 grams: 81.2 %


### Output Analysis

We can see here Accuracy of unigram for testing in Italian Data gets us 65.2 % accuracy.
for BiGram, the accuracy reduces 79.9 %
for TriGram, the accuracy is also 78.9 %

On the other hand, The accuracy for Spanish test data on Spanish model is 65.9 for unigram, 61.7% for bigram and 81.2 % for trigram. Here unigram performances is not good at all. 

So, in this model testing, Trigram for Spanish beats Italian, However Unigram and Bigram in Italian Language is better here. Finally, we can conclude that Italian can be easily distingusable.

### Harder to Distinguishable

Based on our experiment, overall, English vs French accuracy is much higher than the pair of Italian vs Spanish. So we can conclude here that Italian vs Spanish language is harder to distinguish.